### IMPORTS

In [2]:
import pandas as pd
import numpy as np
import pandas_gbq
import datetime
from dateutil.relativedelta import relativedelta
from Roster import *
from gspread_pandas import Spread, conf

FileNotFoundError: [Errno 2] No such file or directory: 'PedidosYa-6e661fd93faf.json'

### CREDENCIALES

In [ ]:
cred = conf.get_config('C:\\Users\\micaela.fuchs\\Anaconda', 'PedidosYa-6e661fd93faf.json')

### CONSTANTES

In [ ]:
# Fechas
today = datetime.date.today()
ilm = str((today - relativedelta(months=1)).replace(day=1))
flm = str(today - relativedelta(days=today.day))

### GOOGLE SHEETS

In [143]:
# Roster
sheet_id = '1M2lrZ-5ZujV0cB3_2MxYqIjWLKicBkyxZzi9r9aUqIU'
wks_name = 'Roster'
sheet = Spread(sheet_id, wks_name, config=cred)
roster = sheet.sheet_to_df(index=0,header_rows=1)

In [144]:
# Me quedo con los Sales
sales_roster = roster[roster['Cargo'] == 'Sales'].copy()
sales_roster['Nombre y Apellido'] = sales_roster['Nombre y Apellido'].apply(lambda x: x.upper())

### QUERIES

In [131]:
# 60MB
q_cw = '''SELECT p.salesforce_id AS Grid,
       IFNULL(CAST(p.partner_id AS STRING),'-') AS Id,
       IFNULL(p.partner_name,'-') AS Name,
       IFNULL(p.city.name,'-') AS City,
       IFNULL(a.area_name,'-') AS Area,
       IFNULL(p.business_type.business_type_name,'-') AS Business,
       sfo.Business_Type AS Tipo,
       IFNULL(sfc.Name__c,'-') AS Nombre,
       cw.account_owner AS Owner,
       DATE(cw.closed_date) AS Fecha_Cierre,
       CASE WHEN cw.add_on = 'Exclusivity' THEN 'Si' ELSE 'No' END AS Exclusivity,
       MAX(sfc.Commission) AS Comision,
FROM `peya-bi-tools-pro.il_core.fact_sales_executives_closed_won_v2` AS cw
LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS p ON cw.grid = p.salesforce_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON p.address.area_id = a.area_id
LEFT JOIN `peya-data-origins-pro.cl_salesforce.opportunity` AS sfo ON cw.oportunity_unique_id = sfo.Id
LEFT JOIN `peya-data-origins-pro.cl_salesforce.contract` AS sfc ON sfo.Id = sfc.Id_Opportunity
WHERE DATE(cw.closed_date) BETWEEN DATE('{0}') AND DATE('{1}')
      AND p.country_id = 3
      AND sfo.Business_Type IN ('New Business','Win Back','Franchise Extension','New Bussiness')
GROUP BY 1,2,3,4,5,6,7,8,9,10,11
ORDER BY 8,6,9 DESC'''.format(ilm,flm)

In [132]:
# Descargo la data
hue_cw = pd.io.gbq.read_gbq(q_cw, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████████| 2807/2807 [00:01<00:00, 2006.18rows/s]


In [133]:
# Copio las bases
cw = hue_cw.copy()

### TRABAJO

In [134]:
# Coloco Feudo y Reino
cw['Feudo'] = cw.apply(feudos,axis=1)
cw['Reino'] = cw.apply(reinos,axis=1)

In [135]:
# Mando el Propietario a Mayuscula
cw['Owner'] = cw['Owner'].apply(lambda x: x.upper())

In [136]:
# Coloco la Zona
roster_cw = sales_roster[sales_roster.columns[1:3]].copy()
roster_cw.columns = ['Owner','Feudo_Owner']
cw = cw.merge(roster_cw,on=['Owner'],how='left')
cw['Feudo_Owner'] = cw['Feudo_Owner'].replace([np.nan,np.inf,-np.inf],'-')

In [137]:
# Cambio las fechas a Str
cw['Fecha_Cierre'] = cw['Fecha_Cierre'].astype(str)

### CARGA

In [109]:
# Carga Roster Sales
sheet_id = '1PPmZJNCsGaMcjb_6xsXcn2MLp8PQBjR-TieLOicUGro'
wks_name = 'Roster Sales'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(sales_roster, index=False, sheet=wks_name, replace=True)

{'spreadsheetId': '1PPmZJNCsGaMcjb_6xsXcn2MLp8PQBjR-TieLOicUGro',
 'replies': [{'findReplace': {}}]}

In [140]:
# Carga Crudo CW
sheet_id = '1PPmZJNCsGaMcjb_6xsXcn2MLp8PQBjR-TieLOicUGro'
wks_name = 'Crudo CW'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(cw, index=False, sheet=wks_name, replace=True)

{'spreadsheetId': '1PPmZJNCsGaMcjb_6xsXcn2MLp8PQBjR-TieLOicUGro',
 'replies': [{'findReplace': {'valuesChanged': 8410,
    'rowsChanged': 2807,
    'sheetsChanged': 1,
    'occurrencesChanged': 8410}}]}